## API con token

_El maravilloso mundo de las_ **APIs**
### o EL RETORNO DE JSON


Cuando es una API con token, tenemos que registrarnos y meter una contraseña y usuario

Listado de **APIs** for free: 
https://github.com/public-apis/public-apis

In [28]:
import json
import requests

In [2]:
json_fullpath = "settings_passwords.json"
# settings.json estará en el .gitignore  

In [4]:
with open(json_fullpath, "r+") as outfile:
    json_readed = json.load(outfile)
json_readed

{'CLIENT': 'KZWZYG43Q4LLZQ2RS5JNN1Q3YRCYEZHMKH0ILOZNNBNPM25Y',
 'API_KEY': 'CBUQ5B10L4CNHGLXXM2TUV0AVH341ADAJ3DWTZSUJB0NAD5M'}

**DOCUMENTACIÓN DE LA API**


--> https://developer.foursquare.com/docs/places-api/ <--

** Cómo crear una cuenta de desarrollo **

--> https://developer.foursquare.com/docs/places-api/getting-started/ <--

** Diferentes Endpoints **

--> https://developer.foursquare.com/docs/places-api/endpoints/ <--

### Objetivo: 

#### Descubrir los restaurantes que están cerca de The Bridge a través de la API de foursquare

In [5]:
url = 'https://api.foursquare.com/v2/venues/explore'

# más códigos de categorías
# https://developer.foursquare.com/docs/build-with-foursquare/categories/

# irish pub
irish_rest = "52e81612bcbc57f1066b7a06"

# para sacar las coordenadas podrías usar una API pero también...
# https://www.123coordenadas.com/coordinates/

lat = 40.4226767
lon = -3.6922754

In [49]:
# Lo importante, todas las APIs con token tienen una forma de acceder a ellas. La información pertinente a cada una de ellas viene (o debería venir) en su documentación

params = {"client_id": json_readed["CLIENT"],
  "client_secret": json_readed["API_KEY"],
  "v":"20180323",
  "ll": f'{lat},{lon}',
  "section": "food",
  # options --> food, drinks, coffee, shops, arts, outdoors, sights, trending
  "limit":100
}  

In [50]:
params

{'client_id': 'KZWZYG43Q4LLZQ2RS5JNN1Q3YRCYEZHMKH0ILOZNNBNPM25Y',
 'client_secret': 'CBUQ5B10L4CNHGLXXM2TUV0AVH341ADAJ3DWTZSUJB0NAD5M',
 'v': '20180323',
 'll': '40.4226767,-3.6922754',
 'section': 'food',
 'limit': 100}

In [51]:
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [53]:
# Muestra cantidad ilegible de datos

data
print()

In [54]:
# Para guardar en un archivo local

# Si quiero guardar la info para no tener que hacer más llamadas al servidor, puedo guardarlo en un CSV
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("toda_informacion.csv")

In [55]:
test = pd.read_csv('toda_informacion.csv')
test

,Unnamed: 0,meta,response
0,code,200,NaN
1,requestId,609910221d1a5131ee9ab126,NaN
2,suggestedFilters,NaN,"{'header': 'Tap to show:', 'filters': [{'name'..."
3,suggestedRadius,NaN,642
4,headerLocation,NaN,Chueca
5,headerFullLocation,NaN,"Chueca, Madrid"
6,headerLocationGranularity,NaN,neighborhood
7,query,NaN,food
8,totalResults,NaN,204
9,suggestedBounds,NaN,"{'ne': {'lat': 40.42757324545612, 'lng': -3.68..."


In [74]:
# test.loc[2:10, "response"]        # -> Series
test.loc[2:10, "response":]        # -> DataFrame
# test.loc[2:10, ["response"]]        # -> DataFrame

,response
2,"{'header': 'Tap to show:', 'filters': [{'name'..."
3,642
4,Chueca
5,"Chueca, Madrid"
6,neighborhood
7,food
8,204
9,"{'ne': {'lat': 40.42757324545612, 'lng': -3.68..."
10,"[{'type': 'Recommended Places', 'name': 'recom..."


In [72]:
# El diccionario no tiene las mismas keys para todas las observaciones. Para "code" y "requestId", los valores son meta
# mientras que para el resto de keys(), los valores son response. 

# Como pandas no es capaz de crear un índice común, crea la columna "Unnamed" donde mete todas las keys, y después las columnas "meta" y "response" donde pone los correspondientes valores. Si la key no tiene valor en meta o response, le pone un NaN.

# Ejemplo: code tiene valor para "meta" pero no para "response", y groups tiene valor para "response" pero no para "meta"

print(data["meta"].keys())
print('-' * 50)
print(data["response"].keys())

dict_keys(['code', 'requestId'])
--------------------------------------------------
dict_keys(['suggestedFilters', 'suggestedRadius', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])


In [56]:
print(type(test.iloc[-1:]))
print(type(test.iloc[-1, :]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [57]:
# df.iloc[-1:].response
# df.iloc[-1:]["response"]
df.iloc[-1:]["response"]

groups    [{'type': 'Recommended Places', 'name': 'recom...
Name: response, dtype: object

In [58]:
from pandas.io.json import json_normalize

s_json = json_normalize(test.iloc[10]["response"])
s_json
# test.iloc[-1, -1]
# test["response"].values[-1]

ipykernel_launcher:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


AttributeError: 'str' object has no attribute 'values'

In [43]:
test["response"]

0                                                   NaN
1                                                   NaN
2     {'header': 'Tap to show:', 'filters': [{'name'...
3                                                   642
4                                                Chueca
5                                        Chueca, Madrid
6                                          neighborhood
7                                                  food
8                                                   204
9     {'ne': {'lat': 40.42757324545612, 'lng': -3.68...
10    [{'type': 'Recommended Places', 'name': 'recom...
Name: response, dtype: object

In [59]:
df.to_json('toda_informacion.json')

In [60]:
# Aqui estamos accediendo al diccionario originario (no al dataframe). Por eso podemos llamar a las claves
data["response"].keys()

dict_keys(['suggestedFilters', 'suggestedRadius', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [45]:
# Cantidad ilegible de datos

data["response"] 

55994,
         'lng': -3.6941006877854963}],
       'distance': 173,
       'postalCode': '28004',
       'cc': 'ES',
       'city': 'Madrid',
       'state': 'Madrid',
       'country': 'España',
       'formattedAddress': ['Calle Conde de Xiquena 2 (esquina Prim) (Prim)',
        '28004 Madrid Madrid',
        'España']},
      'categories': [{'id': '4bf58dd8d48988d155941735',
        'name': 'Gastropub',
        'pluralName': 'Gastropubs',
        'shortName': 'Gastropub',
        'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/gastropub_',
         'suffix': '.png'},
        'primary': True}],
      'verified': True,
      'stats': {'tipCount': 0,
       'usersCount': 0,
       'checkinsCount': 0,
       'visitsCount': 0},
      'beenHere': {'count': 0,
       'lastCheckinExpiredAt': 0,
       'marked': False,
       'unconfirmedCount': 0},
      'photos': {'count': 0, 'groups': []},
      'venuePage': {'id': '405854341'},
      'hereNow': {'count': 0, 'summary': '

In [61]:
df2 = pd.DataFrame(data["response"])
df2

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [48]:
data["response"]['groups']

  'referralId': 'e-3-55324cbb498e151e37dd7505-85'},
   {'reasons': {'count': 0,
     'items': [{'summary': 'This spot is popular',
       'type': 'general',
       'reasonName': 'globalInteractionReason'}]},
    'venue': {'id': '4e8dad502c5b1c1d2121d657',
     'name': 'La Senda De Xiquena',
     'contact': {},
     'location': {'address': 'Calle Conde de Xiquena 2 (esquina Prim)',
      'crossStreet': 'Prim',
      'lat': 40.421978260855994,
      'lng': -3.6941006877854963,
      'labeledLatLngs': [{'label': 'display',
        'lat': 40.421978260855994,
        'lng': -3.6941006877854963}],
      'distance': 173,
      'postalCode': '28004',
      'cc': 'ES',
      'city': 'Madrid',
      'state': 'Madrid',
      'country': 'España',
      'formattedAddress': ['Calle Conde de Xiquena 2 (esquina Prim) (Prim)',
       '28004 Madrid Madrid',
       'España']},
     'categories': [{'id': '4bf58dd8d48988d155941735',
       'name': 'Gastropub',
       'pluralName': 'Gastropubs',
       'sho

In [63]:
data["response"]['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

In [64]:
data["response"]['groups'][0]["items"][0].keys()

dict_keys(['reasons', 'venue', 'referralId'])

In [65]:
data["response"]['groups'][0]["items"][0]["venue"]

{'id': '5ac924f0a22db76045072b51',
 'name': 'Charrua',
 'contact': {},
 'location': {'lat': 40.422416,
  'lng': -3.693999,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.422416, 'lng': -3.693999}],
  'distance': 148,
  'postalCode': '28004',
  'cc': 'ES',
  'city': 'Madrid',
  'state': 'Madrid',
  'country': 'España',
  'formattedAddress': ['28004 Madrid Madrid', 'España']},
 'categories': [{'id': '4bf58dd8d48988d1df931735',
   'name': 'BBQ Joint',
   'pluralName': 'BBQ Joints',
   'shortName': 'BBQ',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'photos': {'count': 0, 'groups': []},
 'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []}}

In [66]:
data["response"]['groups'][0]["items"][0]["venue"]['name']

'Charrua'

In [67]:
for rest in data["response"]['groups'][0]["items"]:
    print(rest["venue"]["name"])

Charrua
Pelotari
Cannibal Raw Bar
Doki Doki
la vaca y la huerta
Panod
celicioso
Flavia Restaurante & Bar
New York Burger
Raimunda
Mamá Chicó
Banzai
El Cafetin De Fernando VI
DSTAgE
Habanera
Food Truck
Shuwa Shuwa
Faborit
Al-Mounia
La Tavernetta
Magasand Columela
a.n.E.l. Tapas & Lounge Bar
il Tavolo Verde
Adolfo Dominguez Lounge Cafe
El Supremo
Aarde
Barrutia
Vides
La Lupita
Krachai
Bunny's Deli
Ramsés
Prep' La Crêpe
Mal de Amores
La Candelita
La Giralda IV
Roostiq
Honest Greens
Mad Grill
Thai Garden 2112
99 Sushi Bar
Roots
La Panaderia de Chueca
Trattoria Pulcinella
La Maquina Jorge Juan
Morgana
Alabaster
El Paraguas
La Magdalena De Proust
niMÚ Barquillo
Kuoco 360
La Bien Aparecida
Hot Now
Taj Mahal
García de La Navarra
Tienda de Vinos (El Comunista)
Sr. Ito
Merci
La Garriga
La alacena de Victor Montes (mercado de San Anton)
La Mafia se sienta a la mesa
Café de París, L' Entrecot
La Primera
La Galette 2 - Casa Escondida
Capperi Mercato Italiano
El Perro y La Galleta
Aloha Poké
Platea 

In [68]:
opciones = [rest["venue"]["name"] for rest in data["response"]['groups'][0]["items"]]
opciones

['Charrua',
 'Pelotari',
 'Cannibal Raw Bar',
 'Doki Doki',
 'la vaca y la huerta',
 'Panod',
 'celicioso',
 'Flavia Restaurante & Bar',
 'New York Burger',
 'Raimunda',
 'Mamá Chicó',
 'Banzai',
 'El Cafetin De Fernando VI',
 'DSTAgE',
 'Habanera',
 'Food Truck',
 'Shuwa Shuwa',
 'Faborit',
 'Al-Mounia',
 'La Tavernetta',
 'Magasand Columela',
 'a.n.E.l. Tapas & Lounge Bar',
 'il Tavolo Verde',
 'Adolfo Dominguez Lounge Cafe',
 'El Supremo',
 'Aarde',
 'Barrutia',
 'Vides',
 'La Lupita',
 'Krachai',
 "Bunny's Deli",
 'Ramsés',
 "Prep' La Crêpe",
 'Mal de Amores',
 'La Candelita',
 'La Giralda IV',
 'Roostiq',
 'Honest Greens',
 'Mad Grill',
 'Thai Garden 2112',
 '99 Sushi Bar',
 'Roots',
 'La Panaderia de Chueca',
 'Trattoria Pulcinella',
 'La Maquina Jorge Juan',
 'Morgana',
 'Alabaster',
 'El Paraguas',
 'La Magdalena De Proust',
 'niMÚ Barquillo',
 'Kuoco 360',
 'La Bien Aparecida',
 'Hot Now',
 'Taj Mahal',
 'García de La Navarra',
 'Tienda de Vinos (El Comunista)',
 'Sr. Ito',
 '

Comprobamos que efectivamente están cerca de **The Bridge**

- **Charrua**

https://www.google.com/maps/place/Charr%C3%BAa+Madrid/@40.4213206,-3.6974034,15z/data=!4m5!3m4!1s0x0:0x8673b16903da6e51!8m2!3d40.4223987!4d-3.6940989

- **Pelotari**

https://www.google.com/maps/place/Pelotari/@40.4210322,-3.7011421,15.06z/data=!4m8!1m2!2m1!1spelotari!3m4!1s0xd42289a8d2d74cb:0x2b6420709612ce53!8m2!3d40.4222232!4d-3.6906525

- **Doki doki**

https://www.google.com/maps/place/Restaurante+Doki+Doki/@40.421046,-3.6919977,17z/data=!3m1!4b1!4m5!3m4!1s0xd42289a5313ac17:0x27cfb02077407a64!8m2!3d40.421046!4d-3.689809